In [ ]:
%matplotlib notebook
import mne
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd

### Load the montage of sensor channel locations and set up the files to process.

In [ ]:
montage = mne.channels.read_montage('standard_1020')
#edf_file = 'EEG_TEST_0001_raw.edf'
#log_file = 'log3.csv'
edf_file = os.path.expanduser('~/Desktop/Data/anya_inversion1_raw.edf')
log_file = os.path.expanduser('~/Desktop/Data/anya_log.csv')

In [ ]:
raw = mne.io.read_raw_edf(edf_file, stim_channel='Trigger', eog=['EEG X1-Pz'], 
                          misc=['EEG CM-Pz','EEG X2-Pz','EEG X3-Pz'])
# Rename the channels so they match the standard montage channel names
raw.rename_channels({c:c.replace('EEG ','').replace('-Pz','') for c in raw.ch_names})
raw.set_montage(montage)
eeg_sample_interval_ms = 1/raw.info['sfreq'] * 1000
print(raw.info)
#raw.plot_sensors()

### Find the events

In [ ]:
events = mne.find_events(raw)

In [ ]:
logdf = pd.read_csv(log_file, header=None, names=['client_ts','trigger_ts'])
logdf = (logdf*1000).astype(int)
logdf['bytecode'] = logdf.client_ts % 255 + 1

### Find the optimal fuzzy alignment between the log file and event bytecode sequence

In [ ]:
logn = logdf.shape[0]
eventn = events.shape[0]
if logn > eventn:
    shift_vals = logn - eventn
    err = np.zeros(shift_vals)
    for shift in range(shift_vals):
        err[shift] = np.sqrt(((logdf.bytecode[shift:shift+eventn] - events[:,2])**2).sum())
    shift = np.argmin(err)
else:
    shift = 0
    err = np.sqrt(((logdf.bytecode[shift:shift+eventn] - events[:,2])**2).sum())

### Merge events and log file
Note that the sequence needs to be resorted based on the client timestamp, as logged events could come in out of order. I.e., an errant TCP packet can hit the trigger device after a subsequent packet.

In [ ]:
df = logdf.copy(deep=True).iloc[shift:shift+eventn,:]
df['bytecodeEvent'] = events[:,2]
#(df.bytecode==df.bytecodeEvent).sum()/df.shape[0]
df['ts_event'] = events[:,0]
df.sort_values('client_ts', inplace=True)
df.reset_index(inplace=True, drop=True)
df['delay'] = ((df.trigger_ts - df.client_ts) / eeg_sample_interval_ms).round().astype(int)
df['ts_event_corrected'] = df.ts_event - df.delay

# TODO: estimate network delay from round-trip times and use that to get an accurate client/tirgger clock bias
clock_bias = df.delay.mean()
print('clock_bias = %0.2fms' % clock_bias)

### Synthesize a corrected event sequence
This new sequence takes into account the random delay from one even to the next and the average network delay. Because event trigger packets can arrive out-of-order, they needed to be resorted above to apply the proper delays. But now that everything is corrected, we can resort them based on the event timestamp so mne doesn't complain about a non-chronological event sequence.

In [ ]:
eventdf = pd.DataFrame(columns=['ts','diff','code'])
# clock_bias is in milliseconds-- convert to the EEG time stamps
eventdf.ts = df.ts_event_corrected.values + int(round(clock_bias / eeg_sample_interval_ms))
eventdf['diff'] = 0
eventdf.code = 1
eventdf.drop_duplicates(subset='ts', keep='first', inplace=True)
eventdf = eventdf.sort_values('ts').reset_index(drop=True)
#plt.plot(eventdf.ts)

In [ ]:
raw_no_ref, _ = mne.set_eeg_reference(raw.load_data().filter(l_freq=None, h_freq=45), [])
#raw_no_ref, _ = mne.set_eeg_reference(raw.load_data(), [])
reject = dict(eeg=180e-6) # 180e-6, eog=150e-6)
event_id, tmin, tmax = {'visual': 1}, -0.1, 0.5
epochs_params = dict(events=eventdf.values, event_id=event_id, tmin=tmin, tmax=tmax, reject=reject)
evoked_no_ref = mne.Epochs(raw_no_ref, **epochs_params).average()
del raw_no_ref  # save memory

In [ ]:
title = 'EEG Original reference'
evoked_no_ref.plot(titles={'eeg':'title'}, time_unit='ms')#, picks=['O1','O2','P3','P4'])
evoked_no_ref.plot_topomap(times=[0.075,0.1,0.125,.15,.175], size=1.0, title=title, time_unit='s')

### Sample code for doing frequency analysis

In [ ]:
occ = raw.get_data(['O1','O2'])

In [ ]:
ft = np.fft.rfft(occ)
T = eeg_sample_interval_ms / 1000
xf = np.linspace(0.0, 1.0/(2.0*T), int(np.ceil(occ.shape[1]/2))+1)

In [ ]:
fig = plt.Figure(figsize=(12,6))
plt.plot(xf[100:15000], np.abs(ft[1,100:15000]))

In [ ]:
logdf.head()

In [ ]:
plt.plot(df.client_ts)

In [ ]:
df.head()

In [ ]:
raw.get_data().shape